In [ ]:
root_dir = !git rev-parse --show-toplevel
import os; os.chdir(root_dir[0])

In [ ]:
BASE_URL = 'http://www.atlasbrasil.org.br'

In [ ]:
HEADERS = ''' 
    -H 'Connection: keep-alive'
    -H 'Cache-Control: max-age=0'
    -H 'Upgrade-Insecure-Requests: 1'
    -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'
    -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3'
    -H 'Referer: http://www.atlasbrasil.org.br/2013/pt/consulta/'
    -H 'Accept-Language: en-US,en;q=0.9,pt-BR;q=0.8,pt;q=0.7'
'''.replace('\n', ' ')

In [ ]:
!curl '{BASE_URL}/2013/pt/download/' {HEADERS} --insecure > /tmp/a.html

In [ ]:
import bs4

In [ ]:
soup = bs4.BeautifulSoup(open('/tmp/a.html').read())

In [ ]:
BASE_PATH = soup.find('head').find('base').attrs['href']
BASE_PATH

In [ ]:
downloads = soup.find_all(class_='blue_button')
links = [b.parent.attrs['href'] for b in downloads]
assert  links[0] == 'data/rawData/Indicadores Atlas - RADAR IDHM.xlsx'

In [ ]:
!mkdir -p data/raw

In [ ]:
links_already_have = [
    "Indicadores Atlas - RADAR IDHM.xlsx",
    "DADOS_DO_ATLAS_DESAGREGADOS_POR_COR__SEXO_E_DOMICILIO_-_2000_E_2010_-_FINAL.xlsx",
    "agrupamento_UDHs_para_calculo_no_IBGE_2000_2010.xlsx",
    "atlas2013_dadosbrutos_pt.xlsx",
    "dados_belem.zip",
    "dados_bh.zip",
    "dados_bs.zip",
    "dados_campinas.zip",
    "dados_curitiba.zip",
    "dados_df.zip",
    "dados_florianopolis.zip",
    "dados_fortaleza.zip",
    "dados_goiania.zip",
    "dados_maceio.zip",
    "dados_manaus.zip",
    "dados_natal.zip",
    "dados_pa.zip",
    "dados_petrolina_juazeiro.zip",
    "dados_recife.zip",
    "dados_sl.zip",
    "dados_teresina_timon.zip",
    "dados_vitoria.zip"
]

links_remaining = set(links) - set(links_already_have)
links_remaining = list(links_remaining)

In [ ]:
import urllib
for l in links_remaining[:]:
    url = BASE_URL + BASE_PATH + urllib.parse.quote(l)
    print(url)
    file = l.split('/')[-1]
    !cd data/raw/; curl "{url}" {HEADERS} --insecure -o '{file}'

In [ ]:
print("Hey, listen")

In [ ]:
# !head data/raw/Indicadores\ Atlas\ -\ RADAR\ IDHM.xlsx 

In [ ]:
!pwd

In [ ]:
lines = !ls -1 ../data/raw/*.zip
size = !du -s ../data/raw/

assert int(size[0].split('\t')[0]) > 10000, size
assert len(lines) == 24

## Unzippin all this crap

In [ ]:
import rarfile
import zipfile
from os import listdir, remove, rmdir, rename
from os.path import isfile, isdir, join

In [ ]:
def get_pattern_files(base_path, pattern):
    return [ join(base_path,file) for file in listdir(base_path) if isfile(join(base_path, file)) and pattern in file ]

def get_all_generated_folders(base_path):
    return [ join(base_path,folder) for folder in listdir(data_path) if isdir(join(data_path, folder))]

def extract_all(path_to_files, extract_to_path, extension):
    extractor = zipfile.ZipFile if extension == '.zip' else rarfile.RarFile
    for path_to_file in path_to_files:
        with extractor(path_to_file, 'r') as ref:
            ref.extractall(extract_to_path)

def move_all_files_from_dir_to_target(target_path, dir_path, extension="."):
    files_names = [join(dir_path, filename) for filename in listdir(dir_path)]
    move_all_files_to_base(target_path, files_names)

def move_all_files_to_target(target_path, files_paths):
    files_names = [ file_path.split('/')[-1] for file_path in files_paths]
    for filename, original_path in zip(files_names, files_paths):
        rename(original_path, join(target_path, filename))

def flat_list(l):
    return [item for sublist in l for item in sublist]

Preparing data folder

In [ ]:
%%bash
rm -rf ../data/preprocessed
mkdir -p ../data/preprocessed
mkdir -p ../data/preprocessed/others
cp ../data/raw/* ../data/preprocessed/
shopt -s extglob
cd ../data/preprocessed/ && mv !(dados_*.zip|others) others 

Extracting everything and removing unnecessary files

In [ ]:
data_path = '../data/preprocessed'

zip_files = get_pattern_files(data_path, '.zip')
extract_all(zip_files, data_path, '.zip')
for folder_path in get_all_generated_folders(data_path):
    move_all_files_from_dir_to_base(data_path, folder_path)

In [ ]:
rar_files = get_pattern_files(data_path, '.rar')
extract_all(rar_files, data_path, '.rar')
for folder_path in get_all_generated_folders(data_path):
    move_all_files_from_dir_to_base(data_path, folder_path)

In [ ]:
for file in zip_files + rar_files:
    remove(file)
directories_to_remove = get_all_generated_folders(data_path)
for folder in directories_to_remove:
    rmdir(folder)

Moving each file type to its specific folder

In [ ]:
%%bash
set -e
cd ../data/preprocessed/
mkdir udh; mkdir regional; mkdir regiao_metro;

In [ ]:
udh_files = get_pattern_files(data_path, "UDH")
move_all_files_to_target(join(data_path, "udh"), udh_files)

regional_files = get_pattern_files(data_path, "Regional") + get_pattern_files(data_path, "REGIO")
move_all_files_to_target(join(data_path, "regional"), regional_files)

metro_files = get_pattern_files(data_path, "RM")
move_all_files_to_target(join(data_path, "regiao_metro"), metro_files)

## Check

In [ ]:
lines = !ls -1 ../data/preprocessed/udh/*.shp
size = !du -s ../data/raw/

assert int(size[0].split('\t')[0]) > 10000, size
assert len(lines) == 24

# Convert Excels to parquets

In [ ]:
import glob
excels = glob.glob('../data/preprocessed/udh/*.xlsx')
excels

In [ ]:
excels.remove('../data/preprocessed/udh/agrupamento_UDHs_para_calculo_no_IBGE_2000_2010.xlsx')

In [ ]:
import multiprocessing, glob, pandas as pd
import pyarrow.lib

In [ ]:
def convert_to_parquet(path):
    print(f'converting {path}')
    dfs = pd.read_excel(path, sheet_name=None)
    for sheet_name, df in dfs.items():
        try:
            if df.empty: continue
            df.to_parquet(path.replace('.xlsx', '__' + sheet_name + '.parquet'), index=False)
        except (Exception, pyarrow.lib.ArrowTypeError) as e:
            print('error in:')
            print((path, sheet_name, e))
            return False
    return True

res = map(convert_to_parquet, excels)
all(res)

In [ ]:
!du -sh ../data/preprocessed/udh/*parquet